# 用django封装的方法实现登入登出
- 因为在网上没有找到完整的从注册到登入，再进入界面验证session ID ，最后登出删除session ID的程序。所以在看了各路大神的方法之后写了一个简单的程序。

### 1.创建一个django项目，里面建一个名为uauth的app， 具体方法可以看前面的文章
### 2. 配置文件，setting.py ， __init__.py 这些都和之前的配置一样
### 3. 迁移数据库，不需要自己再创建表
### 4.主工程下urls.py文件

In [ ]:
rom django.conf.urls import url, include
from django.contrib import admin

from uauth import views

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    # 下面这个路由，是成功登入之后，django内置跳转到的地址，但没有匹配方法，需要自己来实现
    url(r'^accounts/profile/', views.myIndex),
    url(r'^uauth/', include('uauth.urls', namespace='uauth')),
]

### 5. 在app uauth文件下创建urls.py文件，具体内容如下

In [ ]:
from django.conf.urls import url
# 为了实现登入，登出，需要导入下面这两个函数
from django.contrib.auth.views import login, logout

from uauth import views

urlpatterns = [
    # 登入的路由，不可改变
    url(r'^accounts/login/$', login),
    # 登出的路由， 不可改变
    url(r'^accounts/logout/$', logout),
    # 注册用户 ，这个路由可以按自己喜好更改
    url(r'^createuser/', views.createUser),
]

### 6. views.py文件中添加方法

In [ ]:
from django.http import HttpResponse, HttpResponseRedirect
from django.shortcuts import render
from django.contrib.auth.models import User
from django.contrib import auth

# Create your views here.


# 注册用户的方法
def createUser(request):
    if request.method == 'GET':
        return render(request, 'register.html')
    if request.method == 'POST':
        username = request.POST.get('username')
        upassword = request.POST.get('password')
        # 创建新的用户
    user = User.objects.create_user(
        username=username,
        password=upassword
    )
    user.is_staff = True
    user.save()
    # 这个返回可以优化，让它直接转到登录页面
    return HttpResponse('创建成功')


# 验证sesstion ID 保证是登录过得用户才能访问
def myIndex(request):
    if request.user.is_authenticated():
        return render(request, 'index.html')
    else:
        # 验证不通过返回登录页面
        return HttpResponseRedirect('/uauth/accounts/login/')

### 7. 自己编写HTML文件
- 在主项目目录下创建templates文件夹，在settings中配置好路径
- 在templates文件夹下再创建一个registration的文件夹，在下面放一个名为login.html文件
- 注意一定不要打错名字，或直接在templates下创建，否则login方法将不能找到页面
- 其他的注册页面和主页，都直接放在templates下

- login.html 中需要主要用户名和密码的name一定要与下面保持一致

> **注意** 在提交post请求是一定要在后面加上{% csrf_token %}

```HTML

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>登录页面</title>
</head>
<body>
     <form action="/uauth/accounts/login/" method="post">
         {% csrf_token %}
         <label for="username">User name:</label>
         <input type="text" name="username" value="" id="username">
         <label for="password">Password:</label>
         <input type="password" name="password" value="" id="password">

        <input type="submit" value="login" />
        <input type="hidden" name="next" value="{{ next|escape }}" />
         <br />
         <a href="/uauth/createuser/">register</a>
 </form>
</body>
</html>
```


- register.html 

```HTML

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>注册页面</title>
</head>
<body>
     <form action="" method="post">
         {% csrf_token %}
         <label for="username">User name:</label>
         <input type="text" name="username" value="" id="username">
         <label for="password">Password:</label>
         <input type="password" name="password" value="" id="password">

        <input type="submit" value="regiset" />
 </form>
</body>
</html>
```

- index.html

```HTML
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>hello</title>
</head>
<body>
    <h4>hello, world!</h4>
    <a href="/uauth/accounts/logout/">logout</a>
</body>
</html>
```

- 如果想看全部代码可以在这里找到[全部代码](https://github.com/beichen0518/myshare)